In [13]:
#Include
using Plots, LightGraphs, SparseArrays, SimpleWeightedGraphs
using Statistics, BenchmarkTools, LinearAlgebra, ProgressMeter
using Distributions, Base.Threads, CSV, StatsBase
using Base.GC, JLD2, FileIO, TSne
plotly();

In [14]:
function tm()
    println("Number of threads = $(nthreads())")

    # sin1(x::Float64) = ccall((:sin, Base.Math.libm), Float64, (Float64,), x)
    # cos1(x::Float64) = ccall((:cos, Base.Math.libm), Float64, (Float64,), x)
    sin1(x::Float64) = ccall(:sin, Float64, (Float64,), x)
    cos1(x::Float64) = ccall(:cos, Float64, (Float64,), x)

    function test1!(y, x)
        # @assert length(y) == length(x)
        for i = 1:length(x)
            y[i] = sin1(x[i])^2 + cos1(x[i])^2
        end
        y
    end

    function testn!(y::Vector{Float64}, x::Vector{Float64})
        # @assert length(y) == length(x)
        Threads.@threads for i = 1:length(x)
            y[i] = sin1(x[i])^2 + cos1(x[i])^2
        end
        y
    end
    n = 10^7
    x = rand(n)
    y = zeros(n)
    @time test1!(y, x)
    @time testn!(y, x)
    @time test1!(y, x)
    @time testn!(y, x);
    flush(stdout)
end

tm()

Number of threads = 8
  0.371176 seconds
  0.143884 seconds (539 allocations: 35.440 KiB)
  0.546685 seconds
  0.135704 seconds (1 allocation: 48 bytes)


In [15]:
function read_tree(filename)
    lines = readlines(open(filename))
    count = Dict()
    for i = 1:length(lines)
        l = split(lines[i]," ")
        count[l[1]] = 1 
        count[l[2]] = 1
    end
    
    n = length(count)
    G = SimpleGraph(n)
    for i = 1:length(lines)
        l = parse.(Int64,split(lines[i]," "))
        add_edge!(G, l[1]+1,l[2]+1)
    end
    
    return G
end

read_tree (generic function with 1 method)

In [16]:
function gid(D,w,x,y)
    return 0.5*(D[w,x]+D[w,y]-D[x,y])
end

function metric_to_structure_C(d,tol)
    n,_ = size(d)
    W = zeros(2*n,2*n)
    W[1:n,1:n] = d
    G = SimpleGraph(n)
    
    x = 1
    y = 2
    z = 3
    V = collect(4:n)
    
    nextRoots = collect(2*n:-1:n+1)
    
    C = []
    
    G,W,nextRoots,C = recursive_step_C(G,V,x,y,z,W,1,nextRoots,C,tol)
    
    return C
end

function recursive_step_C(G,V,x,y,z,W,ztype,nextRoots,C,tol = 0.000000001)
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = []
    X2 = []
    Y1 = []
    Y2 = []
    Z1 = []
    Z2 = []
    R1 = []
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    xc = false
    yc = false
    zc = false
    
    for w in V
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1,w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(b-c) > tol
                zc = true
                push!(C,(w,x,y,z))
            end
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1,w)
            else
                push!(Z2,w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(a-c) > tol
                push!(C,(w,x,y,z))
                xc = true
            end
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1,w)
            else
                push!(X2,w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(b-a) > tol
                push!(C,(w,x,y,z))
                yc = false
            end
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1,w)
            else
                push!(Y2,w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
        
    G,W,nextRoots,C = zone1_recurse_C(G,W,R1,r,nextRoots,C,tol)
    if !xc
        G,W,nextRoots,C = zone1_recurse_C(G,W,X1,x,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,X2,x,r,nextRoots,C,tol)
    end
    if !yc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Y1,y,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Y2,y,r,nextRoots,C,tol)
    end
    if !zc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Z1,z,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Z2,z,r,nextRoots,C,tol)
    end
    
    return G,W,nextRoots,C
end

function zone1_recurse_C(G,W,V,x,nextRoots,C,tol = 0.000000001)
    ztype = 1 
    if length(V) == 0
        return G,W,nextRoots,C
    end
    
    if length(V) == 1
        add_edge!(G,x,V[1])
        return G,W,nextRoots,C
    end
    
    y = V[1]
    z = V[2]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    C1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    xc = false
    yc = false
    zc = false
    
    Ṽ = V[3:end]
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(b-c) > tol
                zc = true
                if x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - b) < tol && abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(a-c) > tol
                xc = true
                if x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - a) < tol && abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(b-a) > tol
                yc = true
                if x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - b) < tol && abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    C = append!(C,C1[1])
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
        C = append!(C,C1[i])
    end
        
    G,W,nextRoots,C = zone1_recurse_C(G,W,R1,r,nextRoots,C,tol)
    if !xc
        G,W,nextRoots,C = zone1_recurse_C(G,W,X1,x,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,X2,x,r,nextRoots,C,tol)
    end
    if !yc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Y1,y,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Y2,y,r,nextRoots,C,tol)
    end
    if !zc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Z1,z,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Z2,z,r,nextRoots,C,tol)
    end
    
    return G,W,nextRoots,C
end

function zone2_recurse_C(G,W,V,x,y,nextRoots,C,tol = 0.000000001)
    ztype = 2
    if length(V) == 0
        return G,W,nextRoots,C
    end

    z = V[1]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    C1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    xc = false
    yc = false
    zc = false
    
    Ṽ = V[2:end]
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(b-c) > tol
                zc = true
                if x > n && y > n
                    push!(C1[Threads.threadid()],(w,2,1,z))
                    push!(C1[Threads.threadid()],(w,3,1,z))
                    push!(C1[Threads.threadid()],(w,1,2,z))
                    push!(C1[Threads.threadid()],(w,3,2,z))
                    push!(C1[Threads.threadid()],(w,1,3,z))
                    push!(C1[Threads.threadid()],(w,2,3,z))
                elseif y > n
                    push!(C1[Threads.threadid()],(w,x,1,z))
                    push!(C1[Threads.threadid()],(w,x,2,z))
                    push!(C1[Threads.threadid()],(w,x,3,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,x,u,z))
                    end
                elseif x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(c-a) > tol
                xc = true
                if x > n && y > n
                    push!(C1[Threads.threadid()],(w,2,1,z))
                    push!(C1[Threads.threadid()],(w,3,1,z))
                    push!(C1[Threads.threadid()],(w,1,2,z))
                    push!(C1[Threads.threadid()],(w,3,2,z))
                    push!(C1[Threads.threadid()],(w,1,3,z))
                    push!(C1[Threads.threadid()],(w,2,3,z))
                elseif y > n
                    push!(C1[Threads.threadid()],(w,x,1,z))
                    push!(C1[Threads.threadid()],(w,x,2,z))
                    push!(C1[Threads.threadid()],(w,x,3,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,x,u,z))
                    end
                elseif x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(b-a) > tol
                yc = true
                if x > n && y > n
                    push!(C1[Threads.threadid()],(w,2,1,z))
                    push!(C1[Threads.threadid()],(w,3,1,z))
                    push!(C1[Threads.threadid()],(w,1,2,z))
                    push!(C1[Threads.threadid()],(w,3,2,z))
                    push!(C1[Threads.threadid()],(w,1,3,z))
                    push!(C1[Threads.threadid()],(w,2,3,z))
                elseif y > n
                    push!(C1[Threads.threadid()],(w,x,1,z))
                    push!(C1[Threads.threadid()],(w,x,2,z))
                    push!(C1[Threads.threadid()],(w,x,3,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,x,u,z))
                    end
                elseif x > n
                    push!(C1[Threads.threadid()],(w,1,y,z))
                    push!(C1[Threads.threadid()],(w,2,y,z))
                    push!(C1[Threads.threadid()],(w,3,y,z))
                    for u in Ṽ
                        push!(C1[Threads.threadid()],(w,u,y,z))
                    end
                else
                    push!(C1[Threads.threadid()],(w,x,y,z))
                end
            end
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    C = append!(C,C1[1])
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
        C = append!(C,C1[i])
    end
        
    G,W,nextRoots,C = zone1_recurse_C(G,W,R1,r,nextRoots,C,tol)
    if !xc
        G,W,nextRoots,C = zone1_recurse_C(G,W,X1,x,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,X2,x,r,nextRoots,C,tol)
    end
    if !yc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Y1,y,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Y2,y,r,nextRoots,C,tol)
    end
    if !zc
        G,W,nextRoots,C = zone1_recurse_C(G,W,Z1,z,nextRoots,C,tol)
        G,W,nextRoots,C = zone2_recurse_C(G,W,Z2,z,r,nextRoots,C,tol)
    end

    return G,W,nextRoots,C
end

zone2_recurse_C (generic function with 2 methods)

In [17]:
function tree_proj(D,Z2,tol)
    error = 0
    maxE = 0
    n,n = size(D)
    Consts = keys(Z2)
    for (w,i,j,k) in Consts
        a = 2*gid(D,w,i,j)
        b = 2*gid(D,w,j,k)
        c = 2*gid(D,w,i,k)
        if a == maximum([a,b,c])
            d = b-c
            error += abs(d)
            b = b - d/2
            c = c + d/2
        elseif b == maximum([a,b,c])
            d = a-c
            error += abs(d)
            a = a - d/2
            c = c + d/2
        else
            d = a-b
            error += abs(d)
            a = a - d/2
            b = b + d/2
        end
        
        if abs(d) < tol
            delete!(Z2,(w,i,j,k))
        end
        
        if abs(d) > maxE
            maxE = abs(d)
        end
        
        ad = 2*gid(D,w,i,j) - a
        bd = 2*gid(D,w,j,k) - b
        cd = 2*gid(D,w,i,k) - c
                
        D[w,i] -= ad/3
        D[w,j] -= ad/3
        D[i,w] -= ad/3
        D[j,w] -= ad/3
        D[i,j] += ad/3
        D[j,i] += ad/3
                    
        D[w,k] -= bd/3
        D[w,j] -= bd/3
        D[k,w] -= bd/3
        D[j,w] -= bd/3
        D[k,j] += bd/3
        D[j,k] += bd/3

        D[w,i] -= cd/3
        D[w,k] -= cd/3
        D[i,w] -= cd/3
        D[k,w] -= cd/3
        D[i,k] += cd/3
        D[k,i] += cd/3
    end
    
    return D,maxE,error,Z2
end

function tree_proj_full(D,w=rand(1:n))
    error = 0
    maxE = 0
    n,n = size(D)
    Δ = zeros(n,n)
    for i = 1:n
        for j=1:i-1
            for k=1:j-1
                a = gid(D,w,i,j)
                b = gid(D,w,j,k)
                c = gid(D,w,i,k)
                d = 0
                if a == maximum([a,b,c])
                    d = b-c
                    error += abs(d)
                    b = b - d/2
                    c = c + d/2
                elseif b == maximum([a,b,c])
                    d = a-c
                    error += abs(d)
                    a = a - d/2
                    c = c + d/2
                else
                    d = a-b
                    error += abs(d)
                    a = a - d/2
                    b = b + d/2
                end
                
                if abs(d) > maxE
                    maxE = abs(d)
                end
                
                ad = gid(D,w,i,j) - a
                bd = gid(D,w,j,k) - b
                cd = gid(D,w,i,k) - c
                
                D[w,i] -= ad/3
                D[w,j] -= ad/3
                D[i,w] -= ad/3
                D[j,w] -= ad/3
                D[i,j] += ad/3
                D[j,i] += ad/3
                
                D[w,k] -= bd/3
                D[w,j] -= bd/3
                D[k,w] -= bd/3
                D[j,w] -= bd/3
                D[k,j] += bd/3
                D[j,k] += bd/3
                
                D[w,i] -= cd/3
                D[w,k] -= cd/3
                D[i,w] -= cd/3
                D[k,w] -= cd/3
                D[i,k] += cd/3
                D[k,i] += cd/3
            end
        end
    end
    
    @show(error,maxE)
    
    return D,maxE
end

function enumerate_paths(s)
    P = Array{Any,1}(undef,size(s.parents, 1))
    Threads.@threads for v = 1:size(s.parents, 1)
        P[v] = LightGraphs.enumerate_paths(s, v)
    end
    
    return P
end

function h1(G,Z′)
    (n,n) = size(G)
    @inbounds Threads.@threads for i = 1:n
        for j = 1:i-1
            c = min(G[j,i],Z′[j,i])
            G[j,i] -= c
            G[i,j] -= c
            Z′[j,i] -= c            
        end
    end
    return G
end

function h2(F,E,Λ,Π)
    @inbounds Threads.@threads for i = 1:n
        for j = 1:i-1
            δ1 = (F[j,i]+E[j,i])/2
            δ2 = (F[j,i]-E[j,i])/2
            c1 = min(δ1,Λ[j,i])
            c2 = min(δ2,Π[j,i])
            E[j,i] = E[j,i] - c1 + c2
            E[i,j] = E[i,j] - c1 + c2
            F[j,i] = F[j,i] - c1 - c2
            F[i,j] = F[i,j] - c1 - c2
            Λ[j,i] -= c1
            Π[j,i] -= c2
        end
    end
end

h2 (generic function with 1 method)

In [19]:
#returns (w,x,y,z) so that gid(D,w,x,y) == gid(D,w,x,z)
function get_constraints(D,w=rand(1:n))
    n,n = size(D)
    C = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    @inbounds Threads.@threads for i = 1:n
        for j = 1:i-1
            for k = 1:j-1
                a = gid(D,w,i,j)
                b = gid(D,w,j,k)
                c = gid(D,w,k,i)
                
                if a == b
                    push!(C[Threads.threadid()],(w,j,i,k))
                end
                
                if c == b
                    push!(C[Threads.threadid()],(w,k,i,j))
                end
                
                if a == c
                    push!(C[Threads.threadid()],(w,i,j,k))
                end
            end
        end
    end
    
    Cp = C[1]
    for i = 2:16
        C = append!(Cp,C[i])
    end
    
    return Cp
end

function tree_proj_have_structure(D,Consts)
    error = 0
    maxE = 0
    n,n = size(D)
    for (w,i,j,k) in Consts
        a = 2*gid(D,w,i,j)
        b = 4*gid(D,w,j,k)
        c = 2*gid(D,w,i,k)
        d = a-c
        error += abs(d)
        a = a - d/2
        c = c + d/2
        
        if abs(d) > maxE
            maxE = abs(d)
        end
        
        d2 = b - a - c
        if d2 < 0
        
        ad = 2*gid(D,w,i,j) - a
        cd = 2*gid(D,w,i,k) - c
                
        D[w,i] -= ad/3
        D[w,j] -= ad/3
        D[i,w] -= ad/3
        D[j,w] -= ad/3
        D[i,j] += ad/3
        D[j,i] += ad/3

        D[w,i] -= cd/3
        D[w,k] -= cd/3
        D[i,w] -= cd/3
        D[k,w] -= cd/3
        D[i,k] += cd/3
        D[k,i] += cd/3
        
    end
    
    return D,maxE
end
end

tree_proj_have_structure (generic function with 1 method)

In [20]:
function BregmanCC(D,W⁻¹,C,γ,tol,filename)
    (n,n) = size(D)
    g = LightGraphs.SimpleGraphs.CompleteGraph(n)
    G = copy(D)
    Z = Dict()
    Z2 = Dict()
    Z′ = zeros(n,n)
    Λ = zeros(n,n)
    Π = zeros(n,n)
    F = -1*γ*(W⁻¹.*C)
    nthds::Int = Threads.nthreads()

    maxD = tol+1
    maxE = tol+1
    count = 1
    z = 1

    # First time through constraints
    #parallel_cyclic_triple_loopE!(G,current_corrections,next_corrections)
    
    while((maxD > tol || maxE > tol) && z > 0 && count < 2000)
        maxD,G,Z,maxE,Z2 = loop(g,G,Z,Z2,Z′,Λ,Π,F,D,W⁻¹,C,γ,count,filename,tol)
        z = length(Z2)
        @show(avg_distortion(G,D)))
        #maxD,G,Z,maxE = loop(g,G,Z,Z′,Λ,Π,F,D,W⁻¹,C,γ,count,filename,tol)
        flush(stdout)
        count+=1
    end
    return (g,count,Z,F,G)
end

#Some random tree constraints and then ocassionally all of them
function loop(g,G,Z,Z2,Z′,Λ,Π,F,D,W⁻¹,C,γ,count,filename,tol)
    for k = 1:10
        #Project onto the cycles we have
        for p in keys(Z)
            z = Z[p]
            l = length(p)
            u = p[1]
            v = p[l]
            d = -1*G[u,v]
            m = W⁻¹[u,v]
            for i = 1:l-1
                d = d + G[p[i], p[i+1]]
                m += W⁻¹[p[i], p[i+1]]
            end
            c = min(d/m,z)
            for i = 1:l-1
                ch = W⁻¹[p[i],p[i+1]]*c
                G[p[i],p[i+1]] -= ch
                G[p[i+1],p[i]] -= ch
            end
            ch = W⁻¹[u,v]*c
            G[u,v] += ch
            G[v,u] += ch
            if z == c || z - c < 1e-14
                delete!(Z,p)
            else
                Z[p] -= c
            end
        end
    end
    
    h1(G,Z′)

    #Find new broken cycles
    FS = LightGraphs.Parallel.floyd_warshall_shortest_paths(g,G) 
    U = FS.dists
    P = enumerate_paths(FS)
    
    maxD2 = norm(U - G,1)
    maxD = maximum(G-U)
    
    #Project onto new broken cycles
    for i = 1:n
        for j = 1:i-1
            if G[j,i] - U[j,i] > 0  
                p = P[j][i] 
                l = length(p)
                u = p[1]
                v = p[l]
                d = -1*G[u,v]
                m = W⁻¹[u,v]
                for k = 1:l-1
                    d = d + G[p[k], p[k+1]]
                    m += W⁻¹[p[k], p[k+1]]
                end
                if d < 0
                    c=d/m
                    for k = 1:l-1
                        G[p[k],p[k+1]] -= W⁻¹[p[k],p[k+1]]*c
                        G[p[k+1],p[k]] -= W⁻¹[p[k],p[k+1]]*c
                    end
                    G[p[1],p[l]] += W⁻¹[p[1],p[l]]*c
                    G[p[l],p[1]] += W⁻¹[p[1],p[l]]*c
                    if haskey(Z,p)
                        Z[p] = Z[p] - c
                    else
                        Z[p] = -1*c
                    end
                end
            end   
        end
    end
    
    P = 0
    U = 0

    flush(stdout)
        
    #Make sure that F = |E|
    E = G - D
    h2(F,E,Λ,Π)
    G = E + D  

    Consts = metric_to_structure_C(G,tol)
    for k in Consts
        if !haskey(Z2,k)
            Z2[k] = 0
        end
    end
    @show(length(Consts))
    
    maxE = 0
    terror = 0
    for i = 1:1
        G2,maxE,terror,Z2 = tree_proj(G,Z2,tol)
        G = G2
    end
    
    @show((length(Z2),maxE,terror))
    flush(stdout)
    
    Consts = 0
    
    maxE2 = 0
    if count %1 == 0
        @time G2,maxE2 = tree_proj_full(G)
        @show(maxE2)
        G = G2
    end
    
    @show((maxD,maxD2,length(Z),count))
    open(filename,"a") do f
        lenz = length(Z)
        if count %100 == 0
            write(f, "maxD = $maxD, maxD2 = $maxD2, length(z) = $lenz, count = $count, maxE = $maxE, maxE2 = $maxE2 \n")
        else
            write(f, "maxD = $maxD, maxD2 = $maxD2, length(z) = $lenz, count = $count, maxE = $maxE \n")
        end
    end

    return maxD,G,Z,maxE,Z2
end

function loop_have_structure(g,G,Z,Z′,Λ,Π,F,D,W⁻¹,C,γ,count,filename)
    for k = 1:10
        #Project onto the cycles we have
        for p in keys(Z)
            z = Z[p]
            l = length(p)
            u = p[1]
            v = p[l]
            d = -1*G[u,v]
            m = W⁻¹[u,v]
            for i = 1:l-1
                d = d + G[p[i], p[i+1]]
                m += W⁻¹[p[i], p[i+1]]
            end
            c = min(d/m,z)
            for i = 1:l-1
                ch = W⁻¹[p[i],p[i+1]]*c
                G[p[i],p[i+1]] -= ch
                G[p[i+1],p[i]] -= ch
            end
            ch = W⁻¹[u,v]*c
            G[u,v] += ch
            G[v,u] += ch
            if z == c || z - c < 1e-14
                delete!(Z,p)
            else
                Z[p] -= c
            end
        end
    end
    
    h1(G,Z′)

    #Find new broken cycles
    FS = LightGraphs.Parallel.floyd_warshall_shortest_paths(g,G) 
    U = FS.dists
    P = enumerate_paths(FS)
    
    maxD2 = norm(U - G,1)
    maxD = maximum(G-U)
    
    #Project onto new broken cycles
    for i = 1:n
        for j = 1:i-1
            if G[j,i] - U[j,i] > 0  
                p = P[j][i] 
                l = length(p)
                u = p[1]
                v = p[l]
                d = -1*G[u,v]
                m = W⁻¹[u,v]
                for k = 1:l-1
                    d = d + G[p[k], p[k+1]]
                    m += W⁻¹[p[k], p[k+1]]
                end
                if d < 0
                    c=d/m
                    for k = 1:l-1
                        G[p[k],p[k+1]] -= W⁻¹[p[k],p[k+1]]*c
                        G[p[k+1],p[k]] -= W⁻¹[p[k],p[k+1]]*c
                    end
                    G[p[1],p[l]] += W⁻¹[p[1],p[l]]*c
                    G[p[l],p[1]] += W⁻¹[p[1],p[l]]*c
                    if haskey(Z,p)
                        Z[p] = Z[p] - c
                    else
                        Z[p] = -1*c
                    end
                end
            end   
        end
    end
    
    P = 0
    U = 0

    flush(stdout)
        
    #Make sure that F = |E|
    E = G - D
    h2(F,E,Λ,Π)
    G = E + D  

    @time Consts = get_constraints(D)
    @show(length(Consts))
    @time for i = 1:10
        G2,maxE = tree_proj_have_structure(G,Consts)
        G = G2
    end
    
    flush(stdout)
    
    Consts = 0
    
    @show((maxD,maxD2,length(Z),count,maxE))
    open(filename,"a") do f
        lenz = length(Z)
        write(f, "maxD = $maxD, maxD2 = $maxD2, length(z) = $lenz, count = $count, maxE = $maxE \n")
    end
    
    return maxD,G,Z,maxE
end

loop_have_structure (generic function with 1 method)

In [21]:
G = read_tree("./../hyperbolics-master/data/edges/bio-diseasome.edges")
n = nv(G)
E = ne(G)
@show((n,E));

(n, E) = (516, 1188)


In [22]:
F = LightGraphs.Parallel.floyd_warshall_shortest_paths(G)
A = F.dists;

In [23]:
filename = "CS-PHD.txt"
open(filename,"w") do f
    write(f,"")
end

0

In [24]:
W⁻¹ = ones(size(A))
A = convert(Array{Float64,2},A)

@time R = BregmanCC(A,W⁻¹,W⁻¹,1,1e-4,filename)

length(Consts) = 89432
(length(Z2), maxE, terror) = (88560, 2.12003514472822, 10771.562450912814)
error = 829939.5968796749
maxE = 2.796862803356375
  4.923317 seconds (38.91 M allocations: 4.054 GiB)
maxE2 = 2.796862803356375
(maxD, maxD2, length(Z), count) = (0.0, 0.0, 0, 1)
length(Consts) = 440387
(length(Z2), maxE, terror) = (426820, 2.1462224915693078, 16726.45364598928)
error = 699241.6742889337
maxE = 2.3521461360871823
  5.154916 seconds (38.87 M allocations: 4.057 GiB)
maxE2 = 2.3521461360871823
(maxD, maxD2, length(Z), count) = (3.2125917830455917, 43490.371002102955, 7666, 2)
length(Consts) = 293548
(length(Z2), maxE, terror) = (561507, 2.5965747621135353, 24186.62993865342)
error = 474117.96643112163
maxE = 1.8950244253967323
  4.128637 seconds (37.41 M allocations: 3.904 GiB)
maxE2 = 1.8950244253967323
(maxD, maxD2, length(Z), count) = (2.5862625662455336, 23756.556045802306, 12991, 3)
length(Consts) = 518499
(length(Z2), maxE, terror) = (942411, 1.6315724248419343, 36025.

length(Consts) = 355116
(length(Z2), maxE, terror) = (907113, 0.2845249979976021, 1710.2484548517716)
error = 4856.879488820214
maxE = 0.214337142928688
  8.601207 seconds (38.26 M allocations: 3.993 GiB, 45.70% gc time)
maxE2 = 0.214337142928688
(maxD, maxD2, length(Z), count) = (0.03398039864043412, 13.069935675296321, 25784, 27)
length(Consts) = 307226
(length(Z2), maxE, terror) = (882975, 0.2766914972942862, 1590.1208210447762)
error = 4544.72351760417
maxE = 0.1993277125044184
  4.389020 seconds (38.16 M allocations: 3.983 GiB)
maxE2 = 0.1993277125044184
(maxD, maxD2, length(Z), count) = (0.03156105869687931, 13.874313259111858, 26061, 28)
length(Consts) = 375017
(length(Z2), maxE, terror) = (869355, 0.26647865628172873, 1482.7348490844806)
error = 4263.8160949629155
maxE = 0.18527568857928234
  8.762219 seconds (38.08 M allocations: 3.974 GiB, 46.56% gc time)
maxE2 = 0.18527568857928234
(maxD, maxD2, length(Z), count) = (0.013753492674092538, 11.183496731290182, 25758, 29)
length

error = 1494.3653254934454
maxE = 0.13079369062090085
  4.254722 seconds (38.29 M allocations: 3.996 GiB)
maxE2 = 0.13079369062090085
(maxD, maxD2, length(Z), count) = (0.009857761263075915, 8.753936823304773, 28449, 52)
length(Consts) = 241026
(length(Z2), maxE, terror) = (480239, 0.12417744180830592, 406.7090197177394)
error = 1434.349847891163
maxE = 0.13074070232653945
  4.649872 seconds (37.45 M allocations: 3.908 GiB)
maxE2 = 0.13074070232653945
(maxD, maxD2, length(Z), count) = (0.0020548561337285776, 8.008933541941024, 28448, 53)
length(Consts) = 159167
(length(Z2), maxE, terror) = (464694, 0.11919021083831094, 385.74026195465655)
error = 1382.5930068300072
maxE = 0.131150850672195
  4.656872 seconds (38.28 M allocations: 3.995 GiB)
maxE2 = 0.131150850672195
(maxD, maxD2, length(Z), count) = (0.005386187218737071, 8.493954618134884, 28776, 54)
length(Consts) = 121713
(length(Z2), maxE, terror) = (449443, 0.11528219592541156, 366.72999950944865)
error = 1329.980776379368
maxE = 

error = 630.3138418479436
maxE = 0.0745259072142821
  4.757114 seconds (37.66 M allocations: 3.931 GiB)
maxE2 = 0.0745259072142821
(maxD, maxD2, length(Z), count) = (0.003562590321087633, 6.810660081107503, 29998, 77)
length(Consts) = 85625
(length(Z2), maxE, terror) = (237166, 0.08313050059184901, 145.3040126623197)
error = 603.2536061373654
maxE = 0.07155295337070378
  4.656907 seconds (37.96 M allocations: 3.961 GiB)
maxE2 = 0.07155295337070378
(maxD, maxD2, length(Z), count) = (0.0011819573712434917, 6.09793238359405, 30010, 78)
length(Consts) = 95804
(length(Z2), maxE, terror) = (232774, 0.08284864520881641, 141.04689960862382)
error = 581.5476653636376
maxE = 0.06865801734317767
  8.493572 seconds (37.72 M allocations: 3.936 GiB, 45.37% gc time)
maxE2 = 0.06865801734317767
(maxD, maxD2, length(Z), count) = (0.0030599494480600953, 5.9147445214764955, 29761, 79)
length(Consts) = 84018
(length(Z2), maxE, terror) = (227788, 0.08251644468016961, 135.75996126431605)
error = 560.4598248

error = 281.43390434984144
maxE = 0.025006226454675318
  4.686749 seconds (38.25 M allocations: 3.992 GiB)
maxE2 = 0.025006226454675318
(maxD, maxD2, length(Z), count) = (0.0006525059516881981, 5.292155654216614, 30729, 102)
length(Consts) = 46132
(length(Z2), maxE, terror) = (166060, 0.03658511115001284, 65.9666721984949)
error = 282.28324656061017
maxE = 0.022455177197539644
  4.324862 seconds (38.68 M allocations: 4.036 GiB)
maxE2 = 0.022455177197539644
(maxD, maxD2, length(Z), count) = (0.0009780470307685363, 5.399974322650815, 31128, 103)
length(Consts) = 42678
(length(Z2), maxE, terror) = (159860, 0.03662014800974722, 63.87841837255926)
error = 268.89403194141966
maxE = 0.02217418642578295
  8.138475 seconds (37.43 M allocations: 3.906 GiB, 46.70% gc time)
maxE2 = 0.02217418642578295
(maxD, maxD2, length(Z), count) = (0.0017826420477256022, 5.666481547300979, 31787, 104)
length(Consts) = 39203
(length(Z2), maxE, terror) = (127569, 0.03788381515077077, 57.486732010452855)
error = 

...

length(Consts) = 91
(length(Z2), maxE, terror) = (28, 0.00041656956148017343, 0.0072674374665546715)
error = 0.8811250235373453
maxE = 0.0003546742111053547
  4.669030 seconds (37.98 M allocations: 3.964 GiB)
maxE2 = 0.0003546742111053547
(maxD, maxD2, length(Z), count) = (3.162082097496466e-6, 4.60560666252969, 53219, 1552)
length(Consts) = 91
(length(Z2), maxE, terror) = (28, 0.00041573842398179295, 0.0072611785115459515)
error = 0.8765976813866859
maxE = 0.00035473009268027766
  4.839341 seconds (37.72 M allocations: 3.936 GiB)
maxE2 = 0.00035473009268027766
(maxD, maxD2, length(Z), count) = (3.160420231296257e-6, 4.605442371302537, 53258, 1553)
length(Consts) = 91
(length(Z2), maxE, terror) = (28, 0.000415278914656092, 0.007252869853916044)
error = 0.8616482897816127
maxE = 0.00035466857669685936
  8.610567 seconds (37.66 M allocations: 3.930 GiB, 46.72% gc time)
maxE2 = 0.00035466857669685936
(maxD, maxD2, length(Z), count) = (3.0135205930648112e-6, 4.606905375915131, 53629, 1554)

length(Consts) = 79
(length(Z2), maxE, terror) = (28, 0.00040565455193508626, 0.006876840797752415)
error = 0.7537032742689974
maxE = 0.00035028667242364975
  8.557492 seconds (37.31 M allocations: 3.894 GiB, 44.39% gc time)
maxE2 = 0.00035028667242364975
(maxD, maxD2, length(Z), count) = (9.654026862371268e-6, 4.604846639148005, 53908, 1577)
length(Consts) = 79
(length(Z2), maxE, terror) = (28, 0.0004051702815610625, 0.006873320347977829)
error = 0.7852465506078911
maxE = 0.00034936610498093046
  4.929958 seconds (38.24 M allocations: 3.991 GiB)
maxE2 = 0.00034936610498093046
(maxD, maxD2, length(Z), count) = (3.519599967916065e-6, 4.604638624483265, 53932, 1578)
length(Consts) = 79
(length(Z2), maxE, terror) = (28, 0.0004050430899429358, 0.006873582454166138)
error = 0.7439284021143964
maxE = 0.0003504246986949511
  4.750497 seconds (37.35 M allocations: 3.898 GiB)
maxE2 = 0.0003504246986949511
(maxD, maxD2, length(Z), count) = (3.7094391949210603e-6, 4.60484232868266, 54174, 1579)
l

Excessive output truncated after 524309 bytes.

length(Consts) = 79
(length(Z2), maxE, terror) = 

({516, 132870} undirected simple Int64 graph, 2000, Dict{Any,Any}([90, 105, 163]=>5.45959e-6,[154, 395, 414]=>9.71288e-5,[349, 130, 513]=>8.92127e-6,[62, 379, 104]=>7.76546e-6,[148, 277, 237]=>0.14455,[370, 259, 430]=>9.739e-5,[167, 11, 308]=>3.4333e-7,[499, 444, 510]=>1.20102e-9,[70, 10, 513]=>1.91973e-7,[383, 385, 454]=>0.0988925…), [-1.0 3.67657e-7 … 0.378033 0.417694; 3.67657e-7 -1.0 … 0.378033 0.417694; … ; 0.378033 0.378033 … -1.0 3.69282; 0.417694 0.417694 … 3.69282 -1.0], [0.0 1.0 … 9.62197 11.5823; 1.0 -9.06511e-5 … 8.62197 10.5823; … ; 9.62197 8.62197 … 0.0 7.69282; 11.5823 10.5823 … 7.69282 0.0])

In [25]:
Dtree = R[end]
for i = 1:n
    Dtree[i,i] = 0
end

In [26]:
Dtree

516×516 Array{Float64,2}:
  0.0       1.0      11.0       11.0       …  10.0945   9.62197  11.5823 
  1.0       0.0      10.0       10.0           9.09454  8.62197  10.5823 
 11.0      10.0       0.0        0.762168     10.0945   9.62197  11.5823 
 11.0      10.0       0.762168   0.0          10.0945   9.62197  11.5823 
 11.0      10.0       1.0        1.0          10.0945   9.62197  11.5823 
 11.2904   10.2903   11.2904    11.2904    …   5.41167  7.40087   7.21957
 10.2904    9.29035  10.2904    10.2904        4.41167  6.40087   6.21957
 10.0       9.0      10.0       10.0           4.12132  6.11052   5.92922
 10.0       9.0      10.0       10.0           4.12132  6.11052   5.92922
 10.1466    9.14659  10.1466    10.1466        2.62078  6.25711   6.07582
  9.96727   8.96727   9.96727    9.96727   …   2.44146  6.07779   5.89649
 10.4411    9.44109  10.4411    10.4411        2.91528  6.55161   6.37032
  6.0       5.0       7.0        7.0           6.09454  5.62197   7.58231
  ⋮         

In [131]:
function gid(D,w,x,y)
    return 0.5*(D[w,x]+D[w,y]-D[x,y])
end

function metric_to_structure(d)
    n,_ = size(d)
    W = zeros(2*n,2*n)
    W[1:n,1:n] = d
    G = SimpleGraph(n)
    
    x = 1
    y = 2
    z = 3
    V = collect(4:n)
    
    nextRoots = collect(2*n:-1:n+1)
    
    G,W,nextRoots = recursive_step(G,V,x,y,z,W,1,nextRoots)
    
    for e in edges(G)
        if e.src == e.dst
            rem_edge!(G,e.src,e.dst)
        end
    end
    
    return G,W
end

function recursive_step(G,V,x,y,z,W,ztype,nextRoots,tol = 0.000000001)
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = []
    X2 = []
    Y1 = []
    Y2 = []
    Z1 = []
    Z2 = []
    R1 = []
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    for w in V
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1,w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1,w)
            else
                push!(Z2,w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1,w)
            else
                push!(X2,w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1,w)
            else
                push!(Y2,w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
        
    G,W,nextRoots = zone1_recurse(G,W,R1,r,nextRoots)
    G,W,nextRoots = zone1_recurse(G,W,X1,x,nextRoots)
    G,W,nextRoots = zone1_recurse(G,W,Y1,y,nextRoots)
    G,W,nextRoots = zone1_recurse(G,W,Z1,z,nextRoots)
    
    G,W,nextRoots = zone2_recurse(G,W,X2,x,r,nextRoots)
    G,W,nextRoots = zone2_recurse(G,W,Y2,y,r,nextRoots)
    G,W,nextRoots = zone2_recurse(G,W,Z2,z,r,nextRoots)
    
    return G,W,nextRoots
end

function zone1_recurse(G,W,V,x,nextRoots,tol = 0.000000001)
    ztype = 1 
    if length(V) == 0
        return G,W,nextRoots
    end
    
    if length(V) == 1
        add_edge!(G,x,V[1])
        return G,W,nextRoots
    end
    
    y = V[1]
    z = V[2]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    Ṽ = V[3:end]
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(W[w,z] - b) < tol && abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(W[w,z] - a) < tol && abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(W[w,z] - b) < tol && abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
    end
        
        
    G,W,nextRoots = zone1_recurse(G,W,R1p,r,nextRoots)
    G,W,nextRoots = zone1_recurse(G,W,X1p,x,nextRoots)
    G,W,nextRoots = zone1_recurse(G,W,Y1p,y,nextRoots)
    G,W,nextRoots = zone1_recurse(G,W,Z1p,z,nextRoots)
    
    G,W,nextRoots = zone2_recurse(G,W,X2p,x,r,nextRoots)
    G,W,nextRoots = zone2_recurse(G,W,Y2p,y,r,nextRoots)
    G,W,nextRoots = zone2_recurse(G,W,Z2p,z,r,nextRoots)
    
    return G,W,nextRoots
end

function zone2_recurse(G,W,V,x,y,nextRoots,tol = 0.000000001)
    ztype = 2
    if length(V) == 0
        return G,W,nextRoots
    end

    z = V[1]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    Ṽ = V[2:end]
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
    end
        
        
    G,W,nextRoots = zone1_recurse(G,W,R1p,r,nextRoots)
    G,W,nextRoots = zone1_recurse(G,W,X1p,x,nextRoots)
    G,W,nextRoots = zone1_recurse(G,W,Y1p,y,nextRoots)
    G,W,nextRoots = zone1_recurse(G,W,Z1p,z,nextRoots)
    
    G,W,nextRoots = zone2_recurse(G,W,X2p,x,r,nextRoots)
    G,W,nextRoots = zone2_recurse(G,W,Y2p,y,r,nextRoots)
    G,W,nextRoots = zone2_recurse(G,W,Z2p,z,r,nextRoots)

    return G,W,nextRoots
end

zone2_recurse (generic function with 2 methods)

In [27]:
function MAP(Dnew,G)
    n = nv(G)
    map = 0
    for i = 1:n
        N = neighborhood(G,i,1)
        D = Dnew[:,i]
        p = sortperm(D)
        P = Dict()
        for j = 1:n
            P[p[j]] = j
        end
        d = length(N)
        for j = 1:d
            R = P[N[j]]
            a = Set(N)
            b = Set(p[1:R])
            map += length(intersect(a,b))/(d*R)
        end
    end
    
    return map/n
end

function avg_distortion(Dnew,Dold)
    n,n = size(Dnew)
    d = 0
    for i = 1:n
        for j = 1:i-1
            d += abs(Dnew[i,j]-Dold[i,j])/Dold[i,j]
        end
    end
    
    return 2*d/(n*(n-1))
end

avg_distortion (generic function with 1 method)

In [156]:
@time G2,W2 = metric_to_structure(copy(Dtree));
@show(is_connected(G2),nv(G2),ne(G2));
D2 = LightGraphs.Parallel.floyd_warshall_shortest_paths(G2,W2).dists;
@show(avg_distortion(D2[1:n,1:n],A));
@show(MAP(D2[1:n,1:n],G));

  0.023172 seconds (181.66 k allocations: 18.413 MiB)
is_connected(G2) = true
nv(G2) = 1029
ne(G2) = 1028
avg_distortion(D2[1:n, 1:n], A) = 0.1002682849306395
MAP(D2[1:n, 1:n], G) = 0.8857084207340875


In [157]:
@time G2,W2 = metric_to_structure(copy(A));
@show(is_connected(G2),nv(G2),ne(G2));
D2 = LightGraphs.Parallel.floyd_warshall_shortest_paths(G2,W2).dists;
@show(avg_distortion(D2[1:n,1:n],A));
@show(MAP(D2[1:n,1:n],G));

  0.017264 seconds (120.70 k allocations: 15.785 MiB)
is_connected(G2) = true
nv(G2) = 687
ne(G2) = 686
avg_distortion(D2[1:n, 1:n], A) = 0.11052060858113577
MAP(D2[1:n, 1:n], G) = 0.9329687428501389


In [29]:
MAP(Dtree,G)

0.9403575618157879

In [30]:
avg_distortion(Dtree,A)

0.07453977186778714